## Pulling together data - Overview

In this notebook, we:

- load the relevant information from bigquery
- perform a train test split
- move the relevant data to a storage bucket.

## Table of Contents
1. Setup
2. Train Test Split
3. Writing the Bash Scripts
4. Running the Bash Scripts

### 1. Setup


To begin with, let's load in credentials and set up bigquery.

In [239]:
from google.cloud import storage
import os
from sklearn.model_selection import train_test_split
import pandas as pd

credential_path = r"C:\Users\matfl\Documents\MSCA 31009 Project -24078705271d.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path


In [237]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### 2. Train Test Split

Next, let's make the table that we will use for test train split.  Here, we are focusing on art from 5 cultures.  

In [241]:
%%bigquery ttv_split_table
SELECT a.culture, a.object_id
FROM `bigquery-public-data.the_met.objects` a 
WHERE a.culture in ('China', 'Japan', 'American', 'French', 'Italian')

Let's make sure we have a proper split.

In [243]:
ttv_split_table.groupby('culture').count()

,object_id
culture,
American,9239
China,10430
French,8853
Italian,4221
Japan,14746


The values above look right for the object IDs!

From the above, we can write automated bash scripts to pull over each of the items into the appropriate folder.

Below are the train test splits.  I can't actually do them with the function, as I need x and y together for the query writing!  So, I have shuffled the table instead..

In [264]:
ttv_split_table = ttv_split_table.sample(frac=1).reset_index(drop=True)


In [265]:
len(ttv_split_table)/10

4748.9

Here, we create test, validation and train object sets from indexing the shuffled table.

In [274]:
test = ttv_split_table[-4749:]
val = ttv_split_table[-(4749*2):-4749]
trn = ttv_split_table[:-(4749*2)]

In [275]:
len(test)+len(val)+len(trn) == len(ttv_split_table)

True

How many are there in test train and validation?

In [282]:
test.groupby('culture').count().sort_values(by='object_id')

,object_id
culture,
Italian,440
French,871
American,881
China,1103
Japan,1454


In [285]:
trn.groupby('culture').count().sort_values(by='object_id')

,object_id
culture,
Italian,3335
French,7076
American,7444
China,8293
Japan,11843


In [284]:
val.groupby('culture').count().sort_values(by='object_id')

,object_id
culture,
Italian,446
French,906
American,914
China,1034
Japan,1449


Looks close enough, though not perfect.  It wont be perfect anyways, because of the number of images per object varying.  

### 3. Writing the Bash Scripts

Next - writing scripts in bash!  The purpose of these scripts is to move the images from the public gcs storage bucket that hosts them, into our gcs bucket, where we can ensure theyre in the right folder structure for modeling.  Each of these is run with the command:

<code>gsutil cat gs://met-image-bucket/[FILENAME] | gsutil -m cp -r -I gs://met-image-bucket/[DESTINATION NAME]
</code>

I'm gonna write a loop of scripts to write, because I am feeling fancy.  This first look runs through each folder and culture, and writes a script to move in the files, according to the list type.

In [311]:
for k, v in {'trn':trn,'test':test,'val':val}.items():
    for cul in ['Italian', 'French', 'American', 'China', 'Japan']:
        script_write = '\n'.join(v.loc[v['culture']==cul,'object_id'].apply(lambda x: 'gs://gcs-public-data--met/'+str(x)).tolist())
        script_write = script_write.encode('utf8')
        print(k)
        print(len(v.loc[v['culture']==cul,:]))
        print(cul)
        f = open((k+'_'+cul+".sh"), "wb")
        f.write(script_write)
        f.close()
    
    

trn
3335
Italian
trn
7076
French
trn
7444
American
trn
8293
China
trn
11843
Japan
test
440
Italian
test
871
French
test
881
American
test
1103
China
test
1454
Japan
val
446
Italian
val
906
French
val
914
American
val
1034
China
val
1449
Japan


This is a loop that prints the commands to run in gsutil, to move them into the right buckets.

In [1]:
for k in ['trn', 'test', 'val']:
    for cul in ['Italian', 'French', 'American', 'China', 'Japan']:
        print('gsutil cat gs://met-image-bucket/'+k+'_'+cul+".sh"\
              ' | gsutil -m cp -r -I gs://met-image-bucket/'+k+'_pre/'+cul+'/')

gsutil cat gs://met-image-bucket/trn_Italian.sh | gsutil -m cp -r -I gs://met-image-bucket/trn_pre/Italian/
gsutil cat gs://met-image-bucket/trn_French.sh | gsutil -m cp -r -I gs://met-image-bucket/trn_pre/French/
gsutil cat gs://met-image-bucket/trn_American.sh | gsutil -m cp -r -I gs://met-image-bucket/trn_pre/American/
gsutil cat gs://met-image-bucket/trn_China.sh | gsutil -m cp -r -I gs://met-image-bucket/trn_pre/China/
gsutil cat gs://met-image-bucket/trn_Japan.sh | gsutil -m cp -r -I gs://met-image-bucket/trn_pre/Japan/
gsutil cat gs://met-image-bucket/test_Italian.sh | gsutil -m cp -r -I gs://met-image-bucket/test_pre/Italian/
gsutil cat gs://met-image-bucket/test_French.sh | gsutil -m cp -r -I gs://met-image-bucket/test_pre/French/
gsutil cat gs://met-image-bucket/test_American.sh | gsutil -m cp -r -I gs://met-image-bucket/test_pre/American/
gsutil cat gs://met-image-bucket/test_China.sh | gsutil -m cp -r -I gs://met-image-bucket/test_pre/China/
gsutil cat gs://met-image-bucket

I ran bash scripts then to load the files - below are my notes.

5 gb it train 17.5 gb fr train 16 gb us train 22.7 gb china train 55.3 japan train

1.1 gb it test 2 gb fr test 2.8 gb us test 3.7 gb china test 6.3 gb japan test

1.1 gb it val 2.1 gb fr val 1.9 gb us val 3 gb china val gb 7.9 japan val

### 4. Running the Bash Scripts

Now for the slower part - moving and renaming them into the real test and train filders and taking only the images.  I tested it out in another script, and this rename file should rename and move our files.  I'll add a piece in the loop that keeps track of where we are in the loop.

The renaming is important, as it allows us to properly put the test/train/val datasets of each culture in the right folders, to allow using a flow from directory when training the model.

In [19]:
import os
from os import environ
from google.cloud import storage

credential_path = r"C:\Users\matfl\Documents\MSCA 31009 Project -24078705271d.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

storage_client = storage.Client()
client = storage_client
bucket = storage_client.get_bucket('met-image-bucket')



def rename_file(bucket, bucketFolderold,bucketFoldernew, fileName):
    """Rename file in GCP bucket."""
    #making a try and continue loop for if the folder doesn't work
    try:
        blob = bucket.blob(bucketFolderold + fileName)
        bucket.rename_blob(blob,
                       new_name=(bucketFoldernew+fileName.replace('/', '_')))
    except:
        print('failure '+fileName)
    

Here's some text to make the lists of the files to rename and move.

In [5]:
def make_lists(prefix):
    ret = []
    for blob in client.list_blobs('met-image-bucket', prefix=prefix):
        ret.append(blob.name)
        
    ret = [i for i in ret if 'json' not in i]
    ret = [i.replace(prefix,'') for i in ret]
    return ret


At the moment, I feel like just copy pasting rather than writing another loop, so I will be bad and write this 15 times.

In [6]:
French_Test_List = make_lists('test_pre/French/')
French_Val_List = make_lists('val_pre/French/')
French_Trn_List = make_lists('trn_pre/French/')

In [7]:
Italian_Test_List = make_lists('test_pre/Italian/')
Italian_Val_List = make_lists('val_pre/Italian/')
Italian_Trn_List = make_lists('trn_pre/Italian/')

In [8]:
American_Test_List = make_lists('test_pre/American/')
American_Val_List = make_lists('val_pre/American/')
American_Trn_List = make_lists('trn_pre/American/')

I should look at how long these take.

In [10]:
import datetime


print(datetime.datetime.now())
China_Test_List = make_lists('test_pre/China/')
China_Val_List = make_lists('val_pre/China/')
China_Trn_List = make_lists('trn_pre/China/')
print(datetime.datetime.now())

2020-07-21 13:47:26.901668
2020-07-21 13:49:28.926075


In [11]:
print(datetime.datetime.now())
Japan_Test_List = make_lists('test_pre/Japan/')
Japan_Val_List = make_lists('val_pre/Japan/')
Japan_Trn_List = make_lists('trn_pre/Japan/')
print(datetime.datetime.now())

2020-07-21 13:49:28.932060
2020-07-21 13:52:37.160409


In [12]:
len(American_Trn_List)

16824

Now it's time to run these.  Again, I'm gonna be a bad coder and do them one by one rather than a loop, since they'll take some time and this way I can more easily run them one by one.

Would probably be smart to combine the make lists function and this loop but whatever.

In [17]:
for x, i in enumerate(Italian_Test_List[:3]):
    if x == 0:
        print(len(Italian_Test_List))
    rename_file(bucket, 'test_pre/Italian/', 'test/Italian/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())

995
0
2020-07-21 13:54:10.590508


It works!  Going to run it all, and not worry about the first 3 because we expect them to fail since we just ran them.

#### Italian

In [18]:
for x, i in enumerate(Italian_Test_List):
    if x == 0:
        print(len(Italian_Test_List))
    rename_file(bucket, 'test_pre/Italian/', 'test/Italian/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())

995
failure
0
2020-07-21 13:55:10.075539
failure
failure


In [20]:
for x, i in enumerate(Italian_Val_List):
    if x == 0:
        print(len(Italian_Val_List))
    rename_file(bucket, 'val_pre/Italian/', 'val/Italian/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

947
0
2020-07-21 14:01:26.032535


Retuning the time indicator to every 750, which should be like 4 minutes

In [21]:
for x, i in enumerate(Italian_Trn_List):
    if x == 0:
        print(len(Italian_Trn_List))
    rename_file(bucket, 'trn_pre/Italian/', 'trn/Italian/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

7160
0
2020-07-21 14:06:40.338846
750
2020-07-21 14:10:27.578891
1500
2020-07-21 14:14:14.054681
2250
2020-07-21 14:17:53.874945
3000
2020-07-21 14:21:36.882717
3750
2020-07-21 14:25:23.208528
4500
2020-07-21 14:29:03.767205
5250
2020-07-21 14:32:49.016709
6000
2020-07-21 14:36:33.844459
6750
2020-07-21 14:40:17.445634
2020-07-21 14:42:24.311305


#### French

In [22]:
for x, i in enumerate(French_Test_List):
    if x == 0:
        print(len(French_Test_List))
    rename_file(bucket, 'test_pre/French/', 'test/French/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

1942
0
2020-07-21 14:42:38.650867
750
2020-07-21 14:46:26.186356
1500
2020-07-21 14:51:15.711424
2020-07-21 14:55:20.979265


In [23]:
for x, i in enumerate(French_Val_List):
    if x == 0:
        print(len(French_Val_List))
    rename_file(bucket, 'val_pre/French/', 'val/French/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

2024
0
2020-07-21 14:55:21.393084
1000
2020-07-21 15:02:01.662737
2000
2020-07-21 15:07:29.309740
2020-07-21 15:07:36.621439


In [24]:
for x, i in enumerate(French_Trn_List):
    if x == 0:
        print(len(French_Trn_List))
    rename_file(bucket, 'trn_pre/French/', 'trn/French/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

16278
0
2020-07-21 15:07:37.217909
750
2020-07-21 15:11:35.421629
1500
2020-07-21 15:15:49.123948
2250
2020-07-21 15:20:07.015728
3000
2020-07-21 15:24:10.928887
3750
2020-07-21 15:29:01.616687
4500
2020-07-21 15:33:05.342998
5250
2020-07-21 15:37:04.266764
6000
2020-07-21 15:41:02.530331
6750
2020-07-21 15:45:05.099628
7500
2020-07-21 15:49:04.535586
8250
2020-07-21 15:53:04.656727
9000
2020-07-21 15:57:02.764450
9750
2020-07-21 16:01:09.476885
10500
2020-07-21 16:05:12.348624
11250
2020-07-21 16:09:14.638134
12000
2020-07-21 16:13:10.426943
12750
2020-07-21 16:17:09.919827
13500
2020-07-21 16:21:16.126604
14250
2020-07-21 16:25:18.781532
15000
2020-07-21 16:29:19.025476
15750
2020-07-21 16:33:47.383760
2020-07-21 16:36:37.519026


#### Japan

Figured i'd get the big one out of the way.

In [25]:
for x, i in enumerate(Japan_Trn_List):
    if x == 0:
        print(len(Japan_Trn_List))
    rename_file(bucket, 'trn_pre/Japan/', 'trn/Japan/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

34399
0
2020-07-21 16:36:38.042606
750
2020-07-21 16:40:37.700245
1500
2020-07-21 16:44:37.815753
2250
2020-07-21 16:49:20.123622
3000
2020-07-21 16:53:52.597491
3750
2020-07-21 16:57:57.005081
4500
2020-07-21 17:01:59.092949
5250
2020-07-21 17:05:57.944270
6000
2020-07-21 17:09:50.597775
6750
2020-07-21 17:13:40.786236
7500
2020-07-21 17:17:29.701425
8250
2020-07-21 17:21:34.922088
9000
2020-07-21 17:25:24.958626
9750
2020-07-21 17:29:15.199888
10500
2020-07-21 17:33:03.671517
11250
2020-07-21 17:36:53.711724
12000
2020-07-21 17:40:41.887611
12750
2020-07-21 17:44:34.285228
13500
2020-07-21 17:48:23.436384
14250
2020-07-21 17:52:14.260040
15000
2020-07-21 17:56:01.891842
15750
2020-07-21 17:59:48.790111
16500
2020-07-21 18:04:04.163581
17250
2020-07-21 18:08:02.986155
18000
2020-07-21 18:12:00.908261
18750
2020-07-21 18:16:05.330383
19500
2020-07-21 18:20:09.309110
20250
2020-07-21 18:24:14.321558
21000
2020-07-21 18:28:13.685219
21750
2020-07-21 18:32:10.930901
22500
2020-07-21 18:36

In [26]:
for x, i in enumerate(Japan_Test_List):
    if x == 0:
        print(len(Japan_Test_List))
    rename_file(bucket, 'test_pre/Japan/', 'test/Japan/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

4261
0
2020-07-21 19:39:37.322483
750
2020-07-21 19:43:35.854938
1500
2020-07-21 19:47:35.146887
2250
2020-07-21 19:51:32.723479
3000
2020-07-21 19:55:28.522124
3750
2020-07-21 19:59:21.628151
2020-07-21 20:02:05.062156


In [27]:
for x, i in enumerate(Japan_Val_List):
    if x == 0:
        print(len(Japan_Val_List))
    rename_file(bucket, 'val_pre/Japan/', 'val/Japan/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

4616
0
2020-07-21 20:02:05.560812
1000
2020-07-21 20:07:31.359527
2000
2020-07-21 20:12:51.272934
3000
2020-07-21 20:18:09.231285
4000
2020-07-21 20:23:43.907744
2020-07-21 20:27:01.245088


#### China

In [28]:
for x, i in enumerate(China_Val_List):
    if x == 0:
        print(len(China_Val_List))
    rename_file(bucket, 'val_pre/China/', 'val/China/', str(i))
    if x % 1000 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

2348
0
2020-07-21 20:27:01.703462
1000
2020-07-21 20:32:21.919693
2000
2020-07-21 20:37:58.378898
2020-07-21 20:39:56.075492


In [29]:
for x, i in enumerate(China_Test_List):
    if x == 0:
        print(len(China_Test_List))
    rename_file(bucket, 'test_pre/China/', 'test/China/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

2601
0
2020-07-21 20:39:56.696353
750
2020-07-21 20:44:01.835894
1500
2020-07-21 20:48:18.022271
2250
2020-07-21 20:52:31.442583
2020-07-21 20:54:52.857995


In [32]:
for x, i in enumerate(China_Trn_List):
    if x == 0:
        print(len(China_Trn_List))
    rename_file(bucket, 'trn_pre/China/', 'trn/China/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

18385
0
2020-07-22 09:17:54.870913
750
2020-07-22 09:24:37.535691
failure 39668/8.jpg
1500
2020-07-22 09:31:05.305814
2250
2020-07-22 09:35:55.453099
3000
2020-07-22 09:40:53.787670
3750
2020-07-22 09:45:42.475687
4500
2020-07-22 09:50:43.682740
5250
2020-07-22 09:55:32.445037
6000
2020-07-22 10:00:19.483681
failure 44759/26.JPG
6750
2020-07-22 10:05:49.963495
7500
2020-07-22 10:10:39.101240
8250
2020-07-22 10:15:29.001752
9000
2020-07-22 10:20:11.804820
9750
2020-07-22 10:24:46.642867
10500
2020-07-22 10:29:16.665886
11250
2020-07-22 10:33:48.821045
12000
2020-07-22 10:38:49.384217
12750
2020-07-22 10:44:08.171297
13500
2020-07-22 10:48:54.011728
14250
2020-07-22 10:53:23.123097
15000
2020-07-22 10:57:53.038791
15750
2020-07-22 11:02:20.577357
16500
2020-07-22 11:06:48.102189
17250
2020-07-22 11:11:33.904284
18000
2020-07-22 11:16:32.572559
2020-07-22 11:19:09.570384


#### AMERICAN

In [30]:
for x, i in enumerate(American_Val_List):
    if x == 0:
        print(len(American_Val_List))
    rename_file(bucket, 'val_pre/American/', 'val/American/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

2017
0
2020-07-21 21:02:11.896347
750
2020-07-21 21:06:16.996556
1500
2020-07-21 21:10:12.954987
2020-07-21 21:12:58.356978


In [31]:
for x, i in enumerate(American_Test_List):
    if x == 0:
        print(len(American_Test_List))
    rename_file(bucket, 'test_pre/American/', 'test/American/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

2407
0
2020-07-21 21:13:01.569865
750
2020-07-21 21:17:06.972218
1500
2020-07-21 21:21:09.600339
2250
2020-07-21 21:25:10.668527
2020-07-21 21:26:02.984257


In [33]:
for x, i in enumerate(American_Trn_List):
    if x == 0:
        print(len(American_Trn_List))
    rename_file(bucket, 'trn_pre/American/', 'trn/American/', str(i))
    if x % 750 < 1:
        print(x)
        print(datetime.datetime.now())
print(datetime.datetime.now())

16824
0
2020-07-22 11:19:52.724225
750
2020-07-22 11:24:34.434914
failure 107122/4.jpg
1500
2020-07-22 11:29:46.679557
2250
2020-07-22 11:34:18.530223
3000
2020-07-22 11:38:54.984691
3750
2020-07-22 11:43:51.043639
4500
2020-07-22 11:47:41.586006
5250
2020-07-22 11:51:44.294338
6000
2020-07-22 11:55:37.529916
6750
2020-07-22 11:59:28.922067
7500
2020-07-22 12:03:38.580501
8250
2020-07-22 12:07:35.915809
9000
2020-07-22 12:11:35.981574
9750
2020-07-22 12:15:36.202032
10500
2020-07-22 12:19:30.410655
11250
2020-07-22 12:23:25.792702
12000
2020-07-22 12:27:21.870493
12750
2020-07-22 12:31:15.481204
13500
2020-07-22 12:35:12.848188
14250
2020-07-22 12:39:08.711617
15000
2020-07-22 12:43:07.479632
15750
2020-07-22 12:47:00.086391
16500
2020-07-22 12:50:50.000432
2020-07-22 12:52:29.129458
